# <font color="hotpink">Eco_Fertilization</font>

## Weather API
* Using <a href="https://www.weatherbit.io/api/weather-forecast-16-day">Weatherbit | 16 Day Forecast</a>
* Acessing 7 days of forecast data in json format.
* Data requests
    1. temperature (Celsius)
    2. humidity (relative humidity in %)
    3. rainfall (mm)
* API call format: ```https://api.weatherbit.io/v2.0/forecast/daily?city=Raleigh,NC&key=API_KEY&days=5```
* Effective Rainfall: https://www.fao.org/3/S2022E/s2022e03.htm
* Rates of rainfall:
    * Drizzle, very small droplets.
    * Slight (fine) drizzle: Detectable as droplets only on the face, car windscreens and windows.
    * Moderate drizzle: Windows and other surfaces stream with water.
    * Heavy (thick) drizzle: Impairs visibility and is measurable in a raingauge, rates up to 1 mm per hour.
    * Rain, drops of appreciable size and may be described as small to large drops. It is possible to have rain drops within drizzle!
    * Slight rain: Less than 0.5 mm per hour.
    * Moderate rain: Greater than 0.5 mm per hour, but less than 4.0 mm per hour.
    * Heavy rain: Greater than 4 mm per hour, but less than 8 mm per hour.
    * Very heavy rain: Greater than 8 mm per hour.
    * Slight shower: Less than 2 mm per hour.
    * Moderate shower: Greater than 2 mm, but less than 10 mm per hour.
    * Heavy shower: Greater than 10 mm per hour, but less than 50 mm per hour.
    * Violent shower: Greater than 50 mm per hour.
* Weather API Codes/Icons: https://www.weatherbit.io/api/codes 


## Fertilization 
* Heavy and prolonged rain can wash away recently applied fertilizer. It is also important that the water within the first 24 hours (upto 48 hours) be evenly distributed.
* Rainfall of greater than ½ inch (12.7 mm) puts your fertilizer at risk. 
* Urea converts to ammonium ion within 5 to six days by urease enzyme and it takes another 5 to. Six days to convert it in nitrate by soil bacteria . After this plant roots can absorb it properly but all these are also affected by temp, moisture and soil texture.

## Version Info
* Python version: 3.8.10
* requests module version: 2.22.0
* json module version: 2.0.9

In [1]:
import requests as rq
import json as js

# !python3 --version
# print("requests", rq.__version__)
# print("json", js.__version__)

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [13]:
base_url = "https://api.weatherbit.io/v2.0/forecast/daily?"
api_key = "480589e42e7c4352abe4fe25bd398ab0"
city_name = "bangalore"
days = 7

complete_url = "{0}city={1}&key={2}&days={3}".format(base_url, city_name, api_key, days)
print(complete_url, "\n")

response = rq.get(complete_url)
response_code = response.status_code
print("Response Code:", response_code)


if response_code == 200:
    json_obj = response.json()
    with open('weather_data.json', 'w') as file:
        js.dump(json_obj, file, indent = 1, sort_keys = True)
        
    print("City:", json_obj['city_name'], "\n")
    
    prolonged_precip = 0
    prolonged_prob = 0
    heavy_rain_2d = False
    heavy_rain_chance_2d = 0
    precip_2d = 0
    precip_chance_2d = 0
    
    for i in range(days):
        date = json_obj['data'][i]['datetime']
        temp = json_obj['data'][i]['temp']
        rh = json_obj['data'][i]['rh']
        precip = json_obj['data'][i]['precip']
        prob = json_obj['data'][i]['pop']
        w_code = json_obj['data'][i]['weather']['code']
        w_desc = json_obj['data'][i]['weather']['description']
        
        prolonged_precip += precip
        prolonged_prob += prob
        
        count_2d = 0
        if i < 2:
            precip_2d += precip
            precip_chance_2d += prob
            if w_code in [202, 233, 502, 521, 522]:
                heavy_rain_2d = True
                heavy_rain_chance_2d += prob
                count_2d += 1
                heavy_rain_chance_2d //= count_2d
        
        print("Date:", date)
        print("Temperature:", temp)
        print("Relative Humidity:", rh)
        print("Rainfall:", precip)
        print("Probability of Precipitation:", prob)
        print("Weather code:", w_code, "->", w_desc)
        print()
    
    prolonged_prob //= days
    precip_chance_2d //= 2
    
    print("-"*80)
    print("The amount of rain for 2 days, counting today:", precip_2d)
    print("Chances of rain for 2 days, counting today:", precip_chance_2d)
    print()
    
    if heavy_rain_2d:
        print("*"*21, "Warning !!!", "*"*21)
        print("Heavy Rain Chances:", heavy_rain_chance_2d)
        print("Heavy Rainfall puts your fertilizer at risk.")
        print("*"*21, "Warning !!!", "*"*21)
    
    print("Prolonged Precipitation:", prolonged_precip)
    print("Prolonged Precipitation Probability:", prolonged_prob)
    print()
    
    if prolonged_precip > 12.7 and prolonged_prob >= 50:
        print("*"*21, "Warning !!!", "*"*21)
        print("Prolonged Rainfall of greater than 12.7 mm puts your fertilizer at risk.")
        print("*"*21, "Warning !!!", "*"*21)

https://api.weatherbit.io/v2.0/forecast/daily?city=bangalore&key=480589e42e7c4352abe4fe25bd398ab0&days=7 

Response Code: 200
City: Bangalore 

Date: 2022-06-02
Temperature: 27.9
Relative Humidity: 55
Rainfall: 3.125
Probability of Precipitation: 50
Weather code: 201 -> Thunderstorm with rain

Date: 2022-06-03
Temperature: 26.5
Relative Humidity: 57
Rainfall: 3.5
Probability of Precipitation: 55
Weather code: 201 -> Thunderstorm with rain

Date: 2022-06-04
Temperature: 27
Relative Humidity: 60
Rainfall: 2.8125
Probability of Precipitation: 50
Weather code: 803 -> Broken clouds

Date: 2022-06-05
Temperature: 25.5
Relative Humidity: 67
Rainfall: 1.1875
Probability of Precipitation: 30
Weather code: 804 -> Overcast clouds

Date: 2022-06-06
Temperature: 25.1
Relative Humidity: 67
Rainfall: 5.8125
Probability of Precipitation: 65
Weather code: 201 -> Thunderstorm with rain

Date: 2022-06-07
Temperature: 23.3
Relative Humidity: 75
Rainfall: 6.3125
Probability of Precipitation: 70
Weather cod